In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
import datetime
import matplotlib.pyplot as plt
import collections as cc
import seaborn as sns
from collections import Counter
import re
from bs4 import BeautifulSoup
from konlpy.tag import Okt



In [2]:
data_dir='Airfocre_integrated_0725.csv'
score_dir='score_image_0725_normalized.csv'
data=pd.read_csv(data_dir,encoding='cp949')
score=pd.read_csv(score_dir,encoding='utf-8')

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import numpy as np

# NIMA 모델 로드
def load_nima_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.25)(x)
    predictions = Dense(10, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # 사전 학습된 NIMA 가중치 로드
    model.load_weights('weights_mobilenet_aesthetic_0.07.hdf5')
    return model

# 이미지 준비
def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

# 이미지 평가
def score_image(model, img_path):
    img = prepare_image(img_path)
    scores = model.predict(img)[0]
    mean_score = np.dot(scores, np.arange(1, 11))
    return mean_score

# 이미지 평가 수행
def evaluate_images(image_paths):
    model = load_nima_model()
    results = {}
    for img_path in image_paths:
        score = score_image(model, img_path)
        results[img_path] = score
    return results

# 예제 사용법
if __name__ == "__main__":
    image_paths = ['Data/01b7f8442fd16cea51124d129762545c/01b7f8442fd16cea51124d129762545c_5.jpg']
    results = evaluate_images(image_paths)
    for img_path, score in results.items():
        print(f'Image: {img_path}, Score: {score}')


ValueError: Layer count mismatch when loading weights from file. Model expected 105 layers, found 55 saved layers.